# MIMIC-IV query API tutorial

### This notebook shows examples of how to use the cyclops.query API on MIMIC-IV.

* First, setup the MIMIC-IV database according to the instructions in [mimic-code](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `MIMICIVQuerier` accordingly.

## Imports

In [1]:
from sqlalchemy import select

import cyclops.query.util as query_util
from cyclops.query.mimiciv import MIMICIVQuerier

mimic = MIMICIVQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciv-2.0",
    user="postgres",
    password="pwd",
)

2023-01-10 11:40:23,775 INFO cyclops.query.orm - Database setup, ready to run queries!


## Example 1. Get all patient encounters from 2015 (approximate year of care).

In [2]:
encounters = mimic.patient_encounters(years=2015)
encounters.run()
print(f"{len(encounters.data)} rows extracted!")

2023-01-10 11:40:25,133 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:40:25,135 INFO cyclops.utils.profile - Finished executing function run_query in 1.330279 s


69460 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD long title), in the year 2015.

In [3]:
encounters = mimic.patient_encounters(years=[2015])
encounters_schizophrenia = mimic.patient_diagnoses(
    diagnosis_substring="schizophrenia", patients=encounters
)
encounters_schizophrenia.run()
print(f"{len(encounters_schizophrenia.data)} rows extracted!")
encounters_schizophrenia.data["diagnosis_title"].value_counts()

2023-01-10 11:40:25,928 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:40:25,929 INFO cyclops.utils.profile - Finished executing function run_query in 0.760619 s


4731 rows extracted!


Unspecified schizophrenia, unspecified                                     1511
Schizophrenia, unspecified                                                 1431
Paranoid type schizophrenia, unspecified                                    527
Paranoid schizophrenia                                                      381
Paranoid type schizophrenia, chronic                                        279
Paranoid type schizophrenia, chronic with acute exacerbation                174
Unspecified schizophrenia, chronic                                           77
Catatonic schizophrenia                                                      68
Other specified types of schizophrenia, unspecified                          43
Catatonic type schizophrenia, unspecified                                    37
Other schizophrenia                                                          33
Disorganized type schizophrenia, chronic with acute exacerbation             20
Catatonic type schizophrenia, chronic wi

## Example 3. Advanced - uses regexp_match from sqlalchemy. Get all patient encounters with diagnoses (ICD long title contains `schizophrenia` and `chronic` ), in the year 2015.

In [4]:
encounters = mimic.patient_encounters(years=[2015])
diagnoses = mimic.patient_diagnoses(patients=encounters)
subquery = (
    select(diagnoses.query)
    .where(
        diagnoses.query.c.diagnosis_title.regexp_match(
            "(?=.*schizophrenia)(?=.*chronic)"
        )
    )
    .subquery()
)
patients_schizophrenia_chronic = diagnoses
patients_schizophrenia_chronic.query = subquery
patients_schizophrenia_chronic.run()
print(f"{len(patients_schizophrenia_chronic.data)} rows extracted!")

2023-01-10 11:40:27,576 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:40:27,577 INFO cyclops.utils.profile - Finished executing function run_query in 1.606089 s


644 rows extracted!


## Example 4. Advanced - uses cyclops.query.utils.starts_with helper function. Get all patient encounters with diagnoses (starts with `Paranoid` in ICD long title), in the year 2015.

In [5]:
encounters = mimic.patient_encounters(years=[2015])
diagnoses = mimic.patient_diagnoses(patients=encounters)
subquery = (
    select(diagnoses.query)
    .where(query_util.starts_with(diagnoses.query.c.diagnosis_title, "Paranoid"))
    .subquery()
)
patients_schizophrenia = diagnoses
patients_schizophrenia.query = subquery
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")

2023-01-10 11:40:36,607 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:40:36,608 INFO cyclops.utils.profile - Finished executing function run_query in 8.971026 s


1429 rows extracted!


## Example 5. Get all patient encounters with diagnoses (ICD code is F209).

In [6]:
patients = mimic.patients()
patients_f209 = mimic.patient_diagnoses(diagnosis_codes=["F209"], patients=patients)
patients_f209.run()
print(f"{len(patients_f209.data)} rows extracted!")

2023-01-10 11:40:46,239 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:40:46,239 INFO cyclops.utils.profile - Finished executing function run_query in 9.602344 s


1431 rows extracted!


## Example 6. Get all patient encounters with diagnoses (`delirium` in ICD long title).

In [7]:
patients = mimic.patients()
patients_delirium = mimic.patient_diagnoses(
    diagnosis_substring="delirium", patients=patients
)
patients_delirium.run()
print(f"{len(patients_delirium.data)} rows extracted!")

2023-01-10 11:40:47,116 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:40:47,117 INFO cyclops.utils.profile - Finished executing function run_query in 0.858650 s


9000 rows extracted!


In [8]:
patients_delirium.data["diagnosis_title"].value_counts()

Delirium due to conditions classified elsewhere                                  3938
Delirium due to known physiological condition                                    3075
Drug-induced delirium                                                             963
Alcohol withdrawal delirium                                                       373
Alcohol dependence with withdrawal delirium                                       210
Vascular dementia, with delirium                                                  169
Senile dementia with delirium                                                      88
Subacute delirium                                                                  35
Alcohol dependence with intoxication delirium                                      28
Alcohol abuse with intoxication delirium                                           20
Other psychoactive substance use, unspecified with intoxication with delirium      15
Sedative, hypnotic or anxiolytic dependence with withd

## Example 7. Get routine vital signs for patients from year 2015, limit to 100 rows.

In [9]:
encounters = mimic.patient_encounters(years=[2018])
patients_vitals = mimic.events(
    patient_encounters_table=encounters.query, categories="Routine Vital Signs"
)
patients_vitals.run(limit=100)
print(f"{len(patients_vitals.data)} rows extracted!")

2023-01-10 11:42:37,628 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:42:37,630 INFO cyclops.utils.profile - Finished executing function run_query in 110.449480 s


100 rows extracted!


## Example 8. Get hemoglobin lab tests for patients from year 2009, limit to 100 rows.

In [10]:
encounters = mimic.patient_encounters(years=[2009])
patients_hemo_labs = mimic.events(
    categories="labs",
    patient_encounters_table=encounters.query,
    event_names="hemoglobin",
)
patients_hemo_labs.run(limit=100)
print(f"{len(patients_hemo_labs.data)} rows extracted!")

2023-01-10 11:42:39,817 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:42:39,819 INFO cyclops.utils.profile - Finished executing function run_query in 1.730873 s


100 rows extracted!


## Example 9. Get all lab events for patients from year 2010, that match substring "sodium", limit to 100 rows.

In [11]:
encounters = mimic.patient_encounters(years=[2009])
patients_sodium_labs = mimic.events(
    categories="labs",
    patient_encounters_table=encounters.query,
    event_name_substring="sodium",
)
patients_sodium_labs.run(limit=100)
print(f"{len(patients_sodium_labs.data)} rows extracted!")

2023-01-10 11:42:41,187 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:42:41,188 INFO cyclops.utils.profile - Finished executing function run_query in 1.321206 s


100 rows extracted!


## Example 10. Get respiratory events for patients from year 2009, limit to 100 rows.

In [12]:
encounters = mimic.patient_encounters(years=[2009])
patients_respiratory = mimic.events(
    categories="Respiratory", patient_encounters_table=encounters.query
)
patients_respiratory.run(limit=100)
print(f"{len(patients_respiratory.data)} rows extracted!")

2023-01-10 11:44:01,944 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:44:01,946 INFO cyclops.utils.profile - Finished executing function run_query in 80.729833 s


100 rows extracted!


## Example 11. Get heart rate measurements of patients with delirium diagnoses, limit to 1000 rows.

In [13]:
patients = mimic.patients()
encounters = mimic.patient_encounters(patients_table=patients.query)
delirium_encounters = mimic.patient_diagnoses(
    diagnosis_substring="delirium", patients_table=encounters.query
)
patients_delirium_heart_rate = mimic.events(
    event_names="Heart Rate", patient_encounters_table=delirium_encounters.query
)
patients_delirium_heart_rate.run(limit=1000)
print(f"{len(patients_delirium_heart_rate.data)} rows extracted!")

2023-01-10 11:45:19,801 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:45:19,802 INFO cyclops.utils.profile - Finished executing function run_query in 77.812058 s


1000 rows extracted!


## Example 12. Get d_items dimension table, for quick reference.

In [14]:
event_lookup_table = mimic.get_interface(mimic.get_table("event_labels"))
lookup_data = event_lookup_table.run()
print(f"{len(lookup_data)} rows extracted!")

2023-01-10 11:45:19,829 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:45:19,830 INFO cyclops.utils.profile - Finished executing function run_query in 0.019274 s


4014 rows extracted!


## Example 13. Running a raw SQL string.

In [15]:
data = mimic._db.run_query("SELECT * FROM mimiciv_hosp.admissions LIMIT 100")
print(f"{len(data)} rows extracted!")

2023-01-10 11:45:19,842 INFO cyclops.query.orm - Query returned successfully!
2023-01-10 11:45:19,843 INFO cyclops.utils.profile - Finished executing function run_query in 0.006526 s


100 rows extracted!
